In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast

from warnings import simplefilter
simplefilter("ignore")

In [2]:
data = pd.read_csv('all_restaurants_with_route_duration.csv')
data.head()

,name,address,coordinates,rating,reviews,categories,avg_bill,cuisine,assortment,photo_url,route_duration
0,"The бык, демократичный мясной ресторан","Невский проспект, 55","{'lat': 59.932201, 'lon': 30.350266}",4.5,1231.0,['Рестораны'],Средний чек 700 ₽,['Европейская кухня'],"['Пицца', 'Шашлык', 'Бургеры', 'Стейки', 'Том-...",['https://i0.photo.2gis.com/main/branch/38/700...,21.0
1,"Commode, бар-ресторан","Невский проспект, 47","{'lat': 59.932556, 'lon': 30.347396}",4.5,1748.0,"['Рестораны', 'Бары', 'Караоке-залы', 'Банкетн...",Средний чек 1500 ₽,"['Итальянская кухня', 'Японская кухня', 'Европ...","['Стейки', 'Салаты', 'Супы', 'Мидии']",['https://i7.photo.2gis.com/main/branch/38/700...,22.7
2,"Пища династии Минь, китайская закусочная и бар","Короленко, 14","{'lat': 59.942488, 'lon': 30.350266}",4.2,78.0,"['Рестораны', 'Бары', 'Кафе']",Средний чек 1700 ₽,['Китайская кухня'],"['Лапша wok', 'Рис wok', 'Салаты', 'Супы']",['https://i9.photo.2gis.com/main/branch/38/700...,15.0
3,"Salone pasta&bar, итальянский ресторан","набережная реки Фонтанки, 30","{'lat': 59.938734, 'lon': 30.34345}",4.2,177.0,"['Рестораны', 'Бары', 'Доставка еды']",Средний чек 3500 ₽,['Итальянская кухня'],"['Пицца', 'Стейки', 'Салаты', 'Супы', 'Начос',...",['https://i5.photo.2gis.com/main/branch/38/700...,19.2
4,"Тартарбар, ресторан","Виленский переулок, 15","{'lat': 59.940025, 'lon': 30.365968}",4.6,503.0,['Рестораны'],Средний чек 2000 ₽,['Авторская кухня'],"['Стейки', 'Салаты', 'Супы', 'Блюда на гриле']",['https://i8.photo.2gis.com/main/branch/38/700...,2.7


##### Выводим средний чек

In [3]:
def extract_number(text):
    # Ищем число в строке с помощью регулярного выражения
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())  # Возвращаем найденное число
    return None  # Если число не найдено, возвращаем None

In [4]:
data['avg_bill'] = data['avg_bill'].apply(extract_number)

##### Кодируем кухни

In [7]:
# pip install ast

In [8]:
# Функция для преобразования строки в список и извлечения кухонь
def extract_cuisines(cuisine_str):
    try:
        # Преобразуем строку в список
        cuisine_list = ast.literal_eval(cuisine_str)
        # Объединяем элементы списка в строку через запятую
        return ', '.join(cuisine_list)
    except (ValueError, SyntaxError):
        # Если преобразование не удалось, возвращаем исходное значение
        return cuisine_str

# Применяем функцию к столбцу
data['cuisine'] = data['cuisine'].apply(extract_cuisines)
data['categories'] = data['categories'].apply(extract_cuisines)
data['assortment'] = data['assortment'].apply(extract_cuisines)

In [9]:
unique_cuisines = set()
for cuisine_str in data['cuisine']:
    # Разделяем строку по запятой и убираем лишние пробелы
    cuisines = [c.strip() for c in cuisine_str.split(',')]
    # Добавляем кухни в множество уникальных значений
    unique_cuisines.update(cuisines)

# Преобразуем множество в отсортированный список
unique_cuisines = sorted(unique_cuisines)

# Выводим результат
print(f"Число уникальных кухонь: {len(unique_cuisines)}")
unique_cuisines

Число уникальных кухонь: 30


['Авторская кухня',
 'Азиатская кухня',
 'Американская кухня',
 'Веганская кухня',
 'Вегетарианская кухня',
 'Восточная кухня',
 'Вьетнамская кухня',
 'Греческая кухня',
 'Грузинская кухня',
 'Домашняя кухня',
 'Европейская кухня',
 'Израильская кухня',
 'Испанская кухня',
 'Итальянская кухня',
 'Кавказская кухня',
 'Китайская кухня',
 'Корейская кухня',
 'Мексиканская кухня',
 'Немецкая кухня',
 'Паназиатская кухня',
 'Русская кухня',
 'Сербская кухня',
 'Средиземноморская кухня',
 'Тайская кухня',
 'Турецкая кухня',
 'Узбекская кухня',
 'Украинская кухня',
 'Французская кухня',
 'Халяльная кухня',
 'Японская кухня']

##### Создаем ohe столбцы для кухонь

In [10]:
# Создаем столбцы для каждой уникальной кухни и заполняем их нулями
for cuisine in unique_cuisines:
    data[cuisine] = 0

for index, row in data.iterrows():
    # Преобразуем строку в список кухонь, разделяя по запятой
    cuisines = [c.strip() for c in row['cuisine'].split(',')]
    for cuisine in cuisines:
        # Убираем лишние пробелы и проверяем, что кухня есть в unique_cuisines
        cuisine_cleaned = cuisine.strip()
        if cuisine_cleaned in unique_cuisines:
            data.at[index, cuisine_cleaned] = 1

In [13]:
data.head(1)

,name,address,coordinates,rating,reviews,categories,avg_bill,cuisine,assortment,photo_url,...,Русская кухня,Сербская кухня,Средиземноморская кухня,Тайская кухня,Турецкая кухня,Узбекская кухня,Украинская кухня,Французская кухня,Халяльная кухня,Японская кухня
0,"The бык, демократичный мясной ресторан","Невский проспект, 55","{'lat': 59.932201, 'lon': 30.350266}",4.5,1231.0,Рестораны,700,Европейская кухня,"Пицца, Шашлык, Бургеры, Стейки, Том-Ям, Салаты...",['https://i0.photo.2gis.com/main/branch/38/700...,...,0,0,0,0,0,0,0,0,0,0


In [11]:
data.columns

Index(['name', 'address', 'coordinates', 'rating', 'reviews', 'categories',
       'avg_bill', 'cuisine', 'assortment', 'photo_url', 'route_duration',
       'Авторская кухня', 'Азиатская кухня', 'Американская кухня',
       'Веганская кухня', 'Вегетарианская кухня', 'Восточная кухня',
       'Вьетнамская кухня', 'Греческая кухня', 'Грузинская кухня',
       'Домашняя кухня', 'Европейская кухня', 'Израильская кухня',
       'Испанская кухня', 'Итальянская кухня', 'Кавказская кухня',
       'Китайская кухня', 'Корейская кухня', 'Мексиканская кухня',
       'Немецкая кухня', 'Паназиатская кухня', 'Русская кухня',
       'Сербская кухня', 'Средиземноморская кухня', 'Тайская кухня',
       'Турецкая кухня', 'Узбекская кухня', 'Украинская кухня',
       'Французская кухня', 'Халяльная кухня', 'Японская кухня'],
      dtype='object')

In [14]:
data.to_csv('preprocessed_data.csv', index=False, encoding='utf-8')